In [2]:
import pandas as pd
import numpy as np
import re
from math import ceil

In [3]:
concepts_level={'الأساسيات': 1,
 'أنواع البيانات': 2,
 'المتغيرات': 2,
 'التعامل مع الأعداد': 3,
 'التعامل مع النصوص': 3,
 'العوامل': 3,
 'المصفوفات': 3,
 'الدوال': 3,
 'الحلقات': 4,
 'الشروط': 4}

required_concepts = {'الأساسيات': [],
                  'أنواع البيانات': ['الأساسيات'],
                  'المتغيرات': ['الأساسيات'],
                  'التعامل مع الأعداد': ['أنواع البيانات','المتغيرات','الأساسيات'],
                  'التعامل مع النصوص': ['أنواع البيانات','المتغيرات','الأساسيات'],
                  'العوامل': ['أنواع البيانات','المتغيرات','الأساسيات'],
                  'المصفوفات': ['أنواع البيانات','المتغيرات','الأساسيات'],
                  'الدوال': ['أنواع البيانات','المتغيرات','الأساسيات'],
                  'الحلقات': ['العوامل','أنواع البيانات','المتغيرات','الأساسيات'],
                  'الشروط': ['العوامل','أنواع البيانات','المتغيرات','الأساسيات']}


In [ ]:
ex = pd.read_csv('ex_dataset_original.csv')
op = pd.read_csv('operators.csv')
keywords = pd.read_csv('keywords.csv')

In [ ]:
keywords['concepts_level'] = keywords['generalConcept'].apply(lambda x: concepts_level.get(x))
op['concepts_level'] = op['generalConcept'].apply(lambda x: concepts_level.get(x))
generalConcept =keywords.groupby(['generalConcept'])['concepts_level'].first().reset_index()
generalConcept.loc[generalConcept.index.max()+1] = pd.Series({'generalConcept': 'المتغيرات', 'concepts_level':2})

In [ ]:
general_df = pd.DataFrame(columns=['keyword', 'general_concept', 'concept_level'])

In [ ]:
generalConcept

In [ ]:
def remove_comments(code):
    code = re.sub(r'\/\/.*', '// \\n', code)
    code = re.sub(r'\/\*.*?\*\/', '/* */ \\n', code, flags=re.DOTALL)
    code = code.strip()
    code = re.sub(r'^\s*\n', '', code, flags=re.MULTILINE)

    return code

In [ ]:
def count_functions(code):
    pattern = r'\w+\s+\w+\(.*\)\s*{?'
    matches = re.findall(pattern, code)
    return len(matches)

In [ ]:
def remove_main_function(code):
    lines = code.split('\n')
    new_lines = []
    is_main_function = False

    for line in lines:
        if 'int main(' in line:
            is_main_function = True
            new_lines.append('\n')
            continue
        elif 'return' in line and is_main_function:
            new_lines.append('\n')
            is_main_function = False
            continue

        new_lines.append(line)

    new_code = '\n'.join(new_lines)
    return new_code

In [ ]:
def detect_concept(code):

    # Regular expressions for different declarations
    declaration_patterns = {
        'الحلقات': r'(?:for\s*\([^;]+;[^;]+;[^)]+\))|(?:while\s*\([^)]+\))|(?:do\s*\{[^}]+\}\s*while\s*\([^)]+\))',
        'الشروط': r'(?:if\s*\([^)]+\)(?:\s*\{[^}]*\})*)|else\s*if\s*\([^)]+\)(?:\s*\{[^}]*\})*|else(?:\s*\{[^}]*\})*|switch\s*\([^)]+\)(?:\s*\{[^}]*\})*',
        #'المتغيرات': r'(?: (?:float|char|int|double|bool)\s+)+([a-zA-Z_]\w*)\s*(?:\[[^\]]+\])?(?:\s*=\s*(?:[^,;{}()]+|{.*}))?\s*(?:,|\[|\)|;|\{)',
        'أنواع البيانات': r'\b(?:unsigned|signed|const|static|extern|volatile|register|auto|bool|char|short|int|long|float|double|void|string)\b',
        'الأساسيات':r'(cin\s*>>.*|cout\s*<<|//.* ?|\/\*(.|\n)+?\*\/)',
        'التعامل مع الأعداد': r'\b(?:acos|asin|atan|atan2|ceil|cos|cosh|exp|fabs|floor|fmod|frexp|ldexp|log|log10|modf|pow|sin|sinh|sqrt|tan|tanh)\b',
        'التعامل مع النصوص': r'\b(?: string|"?\.(append|substr|length|empty|insert|replace|find))\b',
        'المصفوفات': r'\b\w+\[\d*\w*\]',
        #'العوامل': r'(?:\+=|-=|\*=|/=|<=|>=|==|!=|&&|\|\||%=|<<|>>|>>=|<<=|&=|\|=|\^=|\+\+|--|\+|-|\*|/|%|<|>|&|\||\^|!|~|=)',
        'الدوال': r'\b(?:\w+\s+)+\w+\s*\([^)]*\)\s*(?:const)?\s*(?:{[^}]*})?'
    }
    results = {}

# Extract declarations for each pattern
    for key, pattern in declaration_patterns.items():
        results[key] = len(re.findall(pattern, code))
    return results

In [ ]:
def find_general_concepts(code):
    code_without_comments = remove_comments(code)
    code_without_strings = re.sub(r'\".*?\"', '', code_without_comments)
    new_code = remove_main_function(code_without_strings)

    general_concepts_keywords_count = pd.DataFrame(detect_concept(new_code).items(), columns=['generalConcept', 'count'])

    general_concepts_keywords_count = general_concepts_keywords_count.drop(general_concepts_keywords_count[general_concepts_keywords_count['count'] == 0].index)

    print(general_concepts_keywords_count)
    # function_count = count_functions(new_code)
    # if function_count:
    #     general_concepts_keywords_count = pd.concat([general_concepts_keywords_count, pd.DataFrame({'generalConcept':'الدوال', 'count':function_count, 'concepts_level':3}, index=[0])], ignore_index= True)

    general_concepts_keywords_count['concepts_level'] = general_concepts_keywords_count['generalConcept'].apply(lambda x: concepts_level.get(x))

    total_keywords_count = general_concepts_keywords_count['count'].sum()

    general_concepts = general_concepts_keywords_count['generalConcept'].unique().tolist()
    general_concepts_keywords_count['difficulty'] = (general_concepts_keywords_count['concepts_level'] / 4)  * (general_concepts_keywords_count['count'] / total_keywords_count) *100
    return general_concepts, (general_concepts_keywords_count.difficulty.sum()), total_keywords_count
# divide on len(general_concepts)

In [ ]:
ex['generalConcepts'], ex['difficulty_2'], ex['total_keywords_count'] = zip(*ex['code'].apply(find_general_concepts))

In [ ]:
ex

In [ ]:
ex.difficulty_2.min(), ex.difficulty_2.max()

In [ ]:
min = ex.difficulty_2.min()
max = ex.difficulty_2.max()

In [ ]:
ex['difficulty_2'] = ((ex['difficulty_2'] - min) / (max- min))*100

In [ ]:
ex['difficulty_2']

In [ ]:
# solve_XP = 5

In [ ]:
# ex['XP']=  (ex['difficulty_2'] * solve_XP).astype(int)

In [ ]:
# ex[['generalConcepts','XP' , 'difficulty_2', 'total_keywords_count']]

In [ ]:
def find_time(code):
    non_comment_lines = [line for line in code.split('\n') if not (line.strip().startswith('//') or len(line.strip())<2 ) ]
    return len(non_comment_lines)
ex['required_time'] = ex['code'].apply(find_time)

In [ ]:
ex.required_time.min(), ex.required_time.max()

In [ ]:
ex['mapped_required_time'] = (ex['required_time'] - ex['required_time'].min())*100/(ex['required_time'].max() - ex['required_time'].min())
ex['mapped_required_time'] = ex['difficulty_2']/2 + ex['mapped_required_time']/2

ex

In [ ]:
ex = ex.sort_values('difficulty_2', ignore_index=True).reset_index()

In [ ]:
ex

In [ ]:
ex.mapped_required_time.min() *30 /100

In [ ]:
ex['mean'] = (ex.difficulty_2.cumsum())

In [ ]:
ex['mean'] = ex['mean'] - ex['difficulty_2']
ex.loc[ex['index'] > 0, 'mean'] = ex['mean']/ex['index']
ex['mean'] = ex['mean'] * 100/ ex['difficulty_2'].mean()
ex

In [ ]:
ex['code'][0]

In [ ]:
ex

In [ ]:
##تعديل بناء على المتطلبات
def find_required_concepts_numbers_percentage(generalConcepts):

    concept_difficulty = {x : concepts_level[x] for x in generalConcepts}
    max_value = np.max(list(concept_dificulty.values()))
    concepts_with_max_value = [concept for concept, value in concept_dificulty.items() if value == max_value]
    required_concepts_numbers_percentage = []
    for concept in concepts_with_max_value:
        required_concepts_numbers_percentage.append(np.sum([1 if concepts_level[x] < max_value else 0 for x in required_concepts[concept]])*100/len(list(concepts_level.keys())))
    # max_concept_difficulty =
    return np.max(required_concepts_numbers_percentage)

#find_required_concepts_numbers_percentage(['الأساسيات','التعامل مع الأعداد','الحلقات'])
ex['emptying'] = (ex['generalConcepts'].apply(lambda x: find_required_concepts_numbers_percentage(x)) + ex['mean'])/2

In [ ]:
ex

In [ ]:
ex['previous_problem_count'] = ""
ex['total_problem_count'] = ""
ex['concept_problem_practice'] = ""
for index, row in ex.iterrows():
    current_general_concepts = row['generalConcepts']
    previous_counts = []
    total = []
    concept_problem_practice =[]
    for concept in current_general_concepts:
        previous_count = ex.loc[:index-1, 'generalConcepts'].apply(lambda x: concept in x).sum() +1
        total_count = ex['generalConcepts'].apply(lambda x: concept in x).sum()
        previous_counts.append(previous_count)
        total.append(total_count)
        concept_problem_practice.append((previous_count/total_count)*100)
    ex.at[index, 'previous_problem_count'] = previous_counts
    ex.at[index, 'total_problem_count'] = total
    ex.at[index, 'concept_problem_practice'] = concept_problem_practice

In [ ]:
ex

In [ ]:
ex['concept_emptying1'] = ex.apply(lambda x: [element * x['emptying'] /100 for element in x['concept_problem_practice']], axis=1)

In [ ]:
ex[['emptying', 'concept_problem_practice', 'concept_emptying1']]

In [ ]:
ex['concept_emptying2'] = ex.apply(lambda x: [((1/3)*x['emptying']) + ((2/3)*z) for z in x['concept_problem_practice']], axis=1)

In [ ]:
ex[['difficulty_2','emptying', 'concept_problem_practice', 'concept_emptying1', 'concept_emptying2']]

In [ ]:
ex = ex.drop('difficulty', axis=1)

In [ ]:
ex = ex.drop(['concept_emptying1','concept_problem_practice','previous_problem_count','total_problem_count',
             'mean','total_keywords_count','required_time'], axis=1)

In [ ]:
ex = ex.rename({'difficulty_2': 'difficulty'}, axis=1)

In [ ]:
ex = ex.drop('level_0', axis=1)

In [53]:
ex

,index,Consept,required,img_src,code,output,Explanation,hint,generalConcepts,difficulty,mapped_required_time,emptying,concept_emptying2
0,0,إجراء عمليات حسابية,أكتب برنامج يطلب من المستخدم إدخال خمسة أعداد ...,NaN,\r\n#include <iostream>\r\n\r\nint main() {\r\...,\n\r\n\t\t\t\tEnter n1: 7 \r\n\t\t\t\tEnter n2...,سنحصل على النتيجة التالية إذا قام المستخدم بإد...,"مثال: إذا قام المستخدم بإدخال الأعداد 7 , 4 , ...","[أنواع البيانات, الأساسيات]",0.000000,11.250000,5.000000,"[2.1140939597315436, 2.1140939597315436]"
1,1,إجراء عمليات حسابية,"أكتب برنامج يطلب من المستخدم إدخال رقمين, ثم ي...",NaN,\r\n#include <iostream>\r\n\r\nint main() {\r\...,\r\nEnter first number: 14\r\nEnter second num...,سنحصل على النتيجة التالية إذا قام المستخدم بإد...,NaN,"[أنواع البيانات, الأساسيات]",2.472315,7.486158,5.000000,"[2.56152125279642, 2.56152125279642]"
2,2,إجراء عمليات حسابية,أكتب برنامج يطلب من المستخدم إدخال ثلاث أرقام ...,NaN,\r\n#include <iostream>\r\n\r\nint main() {\r\...,\r\n#include <iostream>\r\n\r\nint main() {\r\...,الطريقة الثالثة لحل التمرين و الحصول على نفس ا...,NaN,"[أنواع البيانات, الأساسيات]",3.626062,9.313031,5.928486,"[3.3184440150693164, 3.3184440150693164]"
3,3,إجراء عمليات حسابية,"أكتب برنامج يطلب من المستخدم إدخال رقمين, ثم ي...",NaN,\r\n#include <iostream>\r\n\r\nint main() {\r\...,\r\n#include <iostream>\r\n\r\nint main() {\r\...,الطريقة الثالثة لحل التمرين و الحصول على نفس ا...,NaN,"[أنواع البيانات, الأساسيات]",3.626062,9.313031,6.526844,"[3.9653239438460273, 3.9653239438460273]"
4,4,إجراء عمليات حسابية,"أكتب برنامج يطلب من المستخدم إدخال رقم الشهر, ...",NaN,\r\n#include <iostream>\r\n\r\nint main() {\r\...,\r\nMonth number: 5\r\nMonth name: May\r\n\t\t,سنحصل على النتيجة التالية إذا قام المستخدم بإد...,مثال: إذا قام المستخدم بإدخال الرقم 5 فستكون ا...,"[الشروط, أنواع البيانات, الأساسيات]",6.798867,53.399433,21.826023,"[7.935407096043097, 9.512477554766821, 9.51247..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,144,التعامل مع المصفوفات,"قم بتعريف دالة إسمها CountOccurrence, مهمتها ا...",NaN,\r\n#include <iostream>\r\n\r\n// هنا قمنا بتع...,\r\n[0] is repeated 2 time(s)\r\n[4] is repeat...,سنحصل على النتيجة التالية عند التشغيل.,مساعدة: يجب تمرير حجم المصفوفة أيضاً للدالة لأ...,"[الحلقات, الشروط, أنواع البيانات, الأساسيات, ا...",86.118980,60.559490,69.341639,"[87.83407429621295, 87.14028228927504, 87.9908..."
145,145,التعامل مع المصفوفات,"قم بتعريف دالة إسمها PrintSecondBiggest, عند ا...",NaN,\r\n#include <iostream>\r\n\r\n// printSecondB...,\r\nSecond biggest number is: 8\r\n\t\t,سنحصل على النتيجة التالية عند التشغيل في حال ك...,NaN,"[الحلقات, الشروط, أنواع البيانات, الأساسيات, ا...",90.406554,68.953277,69.447452,"[88.3559633680158, 87.83561936281235, 88.47353..."
146,146,التعامل مع المصفوفات,"قم بتعريف دالة إسمها PrintSecondLowest, عند اس...",NaN,\r\n#include <iostream>\r\n\r\n// printSecondL...,\r\nSecond lowest number is: 2\r\n\t\t,سنحصل على النتيجة التالية عند التشغيل في حال ك...,NaN,"[الحلقات, الشروط, أنواع البيانات, الأساسيات, ا...",91.128672,69.314336,69.573874,"[88.88472184490226, 88.53782584143329, 88.9631..."
147,147,التعامل مع المصفوفات,أكتب برنامج يقوم بترتيب جميع القيم الموجودة في...,NaN,\r\n#include <iostream>\r\n\r\nint main() {\r\...,\r\nMatrix before sorting \r\n5 2 4 \r\n1 7 3 ...,سنحصل على النتيجة التالية عند التشغيل.,NaN,"[الحلقات, الشروط, أنواع البيانات, الأساسيات, ا...",91.406988,79.453494,69.702265,"[89.41413688567938, 89.2406888839449, 89.45332..."


In [54]:
ex.to_csv('ex_dataset.csv', index= False,  encoding='utf-8-sig')

# Tests

In [356]:
x = 4.2222222
y = 4.55
ceil(x)

(5, 5)

In [116]:
operators = ['<<', ';', '"', '=', "'", '"', '+', '-', '/', '*', '++', '--', '%', '==', '!=', '<=', '>=', '<', '>', '||', '&&', '!', '^', '<<', '>>', '~', '|', '&', '+=', '-=', '*=', '/=', '%=', '<<=', '>>=', '&=', '^=', '|=', '?', ':', '[', ']', '//', ',']

def count_operators(code):
    operator_counts = {}

    for operator in operators:
        count = code.count(operator)
        operator_counts[operator] = count

    return operator_counts

# Example C++ code
cpp_code = '''
#include <iostream>

int main() {
    int a = 5;
    int b = 10;
    int result = a + b;

    std::cout << "The result is: " << result << std::endl;

    return 0;
}
'''

# Count operators in the C++ code
operator_counts = count_operators(cpp_code)

# Print the operator counts
for operator, count in operator_counts.items():
    print(f"{operator}: {count}")


<<: 3
;: 5
": 2
=: 3
': 0
+: 1
-: 0
/: 0
*: 0
++: 0
--: 0
%: 0
==: 0
!=: 0
<=: 0
>=: 0
<: 7
>: 1
||: 0
&&: 0
!: 0
^: 0
>>: 0
~: 0
|: 0
&: 0
+=: 0
-=: 0
*=: 0
/=: 0
%=: 0
<<=: 0
>>=: 0
&=: 0
^=: 0
|=: 0
?: 0
:: 5
[: 0
]: 0
//: 0
,: 0


In [117]:
operators = ['<<', ';', '"', '=', "'", '"', '+', '-', '/', '*', '++', '--', '%', '==', '!=', '<=', '>=', '<', '>', '||', '&&', '!', '^', '<<', '>>', '~', '|', '&', '+=', '-=', '*=', '/=', '%=', '<<=', '>>=', '&=', '^=', '|=', '?', ':', '[', ']', '//', ',']

# Create a sample code string
code = 'int a = 10; int b = 20; int c = a + b;'

# Replace operators with underscores
for operator in operators:
    code = code.replace(operator, '_')

print(code)


int a _ 10_ int b _ 20_ int c _ a _ b_


In [120]:
def remove_main_function(code):
    lines = code.split('\n')
    new_lines = []
    is_main_function = False

    for line in lines:
        if 'int main(' in line:
            is_main_function = True
            new_lines.append('\n')
            continue
        elif 'return' in line and is_main_function:
            new_lines.append('\n')
            is_main_function = False
            continue

        new_lines.append(line)

    new_code = '\n'.join(new_lines)
    return new_code

# Sample code string
code = '''
#include <iostream>

int main() {
    std::cout << "Hello, world!" << std::endl;
    return 0;
}

int add(int a, int b) {
    return a + b;
}
'''

# Remove main function and its return statement
modified_code = remove_main_function(code)
print(modified_code)



#include <iostream>



    std::cout << "Hello, world!" << std::endl;


}

int add(int a, int b) {
    return a + b;
}



In [258]:
def remove_comments(code):
    # Remove single-line comments (//)
    code = re.sub(r'\/\/.*', '// \\n', code)
    code = re.sub(r'\/\*.*?\*\/', '/* */ \\n', code, flags=re.DOTALL)

    # Remove empty lines
    code = re.sub(r'^\s*\n', '', code, flags=re.MULTILINE)
    code = code.strip()
    return code

# Example code
example_code = '''
// This is a comment
int main() {
    // This is another comment
    /* This is a multi-line comment */
    int a = 10;
    return 0;
}
'''

# Remove comments from the code
clean_code = remove_comments(example_code)

# Print the clean code
print(clean_code)

// 
int main() {
    // 
    /* */ 
    int a = 10;
    return 0;
}


In [5]:
# Sample dictionary
my_dict = {'الأساسيات': 1, 'أنواع البيانات': 1, 'المتغيرات': 1, 'التعامل مع الأعداد': 1, 'التعامل مع النصوص': 1, 'العوامل': 1, 'المصفوفات': 1, 'الدوال': 1, 'الحلقات': 1, 'الشروط': 1}

# Convert dictionary to a string
dict_string = str(my_dict)

print(dict_string)
# Convert string to a dictionary
new_dict = eval(dict_string)

print(new_dict['name'])


{'الأساسيات': 1, 'أنواع البيانات': 1, 'المتغيرات': 1, 'التعامل مع الأعداد': 1, 'التعامل مع النصوص': 1, 'العوامل': 1, 'المصفوفات': 1, 'الدوال': 1, 'الحلقات': 1, 'الشروط': 1}


KeyError: 'name'

In [343]:
# Two sample lists
keys = ['name', 'age', 'city']
values = ['John', 30, 'New York']

# Convert lists to a dictionary
my_dict = {key: value for key, value in zip(keys, values)}

print(my_dict)

{'name': 'John', 'age': 30, 'city': 'New York'}


In [349]:
import ast

# String representation of a list
list_string = "['apple', 'banana', 'orange']"

# Convert string to a list
my_list = ast.literal_eval(list_string)

print(my_list[0])

apple


In [15]:
def euclidean_distance(point1, point2):
    point1 = np.array(point1)
    point2 = np.array(point2)
    distance = np.linalg.norm(point1 - point2)
    return distance


In [33]:
user = [80, 50, 3, 4, 4, 4, 3, 4]
problem = [10, 50, 1, 2, 1, 3, 4, 2]
distance = euclidean_distance(user, problem)
user_max = [0] *2 + [1] * (len(user) - 2)
problem_max = [100] *2 + [4] * (len(user) - 2)
distance_max = euclidean_distance(user_max, problem_max)
print(distance)
S = 100 * (1- (distance / distance_max))
print(S)

70.16409338115899
50.45333668130863


In [1]:
my_dict = {'a': 5, 'b': 2, 'c': 8, 'd': 1}

# Convert the dictionary into a list of tuples (key, value)
sorted_tuples = sorted(my_dict.items(), key=lambda x: x[1])

# Create a new dictionary from the sorted list of tuples
sorted_dict = dict(sorted_tuples)

print(sorted_dict)


{'d': 1, 'b': 2, 'a': 5, 'c': 8}


In [4]:
my_dict = {'b': 2, 'a': 1, 'c': 3}

sorted_dict = {k: my_dict[k] for k in sorted(my_dict.keys())}

print(sorted_dict)
print(list(sorted_dict.values()))


{'a': 1, 'b': 2, 'c': 3}
[1, 2, 3]


In [38]:
def calculate_xp(task_difficulty, time_spent, performance):
    base_xp = 10  # Base XP for completing a task

    # Adjust XP based on task difficulty
    difficulty_factor = 10  # Adjust this factor based on difficulty levels
    xp = base_xp * task_difficulty

    # Adjust XP based on time spent
    # time_factor = 0.5  # Adjust this factor based on desired time-to-XP ratio
    # xp *= time_factor * time_spent
    #
    # # Adjust XP based on performance
    # performance_factor = performance / 100.0  # Convert performance to a decimal percentage
    # xp *= performance_factor

    # Apply additional adjustments or factors based on engagement metrics or user-specific variables
    # ...
    return int(xp)  # Convert XP to an integer value
# Example usage
task_difficulty = 30  # Example difficulty value (0.0 to 1.0)
time_spent = 30  # Example time spent in minutes
performance = 80  # Example performance as a percentage

xp = calculate_xp(task_difficulty, time_spent, performance)
print(f"XP Earned: {xp}")


XP Earned: 300


In [97]:
import math

def map_to_sigmoid(x, y):
    normalized_x = (x - 50) / 10
    normalized_y = (y - 50) / 10
    result = 1 / (1 + math.exp(-(normalized_x - normalized_y))) *100
    return result

# Example usage
x = 80
y = 80
sigmoid_value = map_to_sigmoid(x, y)
print(sigmoid_value)

50.0


In [12]:
code = '''\n#include <iostream>\n\nint main() {\n\n\tint a, b, c, minimum;\n\n\tstd::cout << \"Enter a: \";\n\tstd::cin >> a;\n\n\tstd::cout << \"Enter b: \";\n\tstd::cin >> b;\n\n\tstd::cout << \"Enter c: \";\n\tstd::cin >> c;\n\n\tminimum = (a < b) ? a : b;\n\tminimum = (minimum < c) ? minimum : c;\n\n\tstd::cout << \"The min number is: \" << minimum;\n\n\tchar end; std::cin >> end;\n\treturn 0;\n\n}\n\t\t'''

operators = ['!=', '!',  '"', '%', '%=', '&', '&&', '&=', "'", '*', '*=', '+', '++', '+=', ',', '-', '--', '-=', '/', '//', '/=', ':', ';', '<', '<<', '<<=', '<=', '=', '==', '>', '>=', '>>', '>>=', '?', '[', ']', '^', '^=', '|', '|=', '||', '~']

operator_counts = {}

for operator in operators:
    count = len(re.findall(r'\s* {} \s*'.format(re.escape(operator)), code, re.MULTILINE))
    operator_counts[operator] = count

print(operator_counts)

{'!': 0, '!=': 0, '"': 1, '%': 0, '%=': 0, '&': 0, '&&': 0, '&=': 0, "'": 0, '*': 0, '*=': 0, '+': 0, '++': 0, '+=': 0, ',': 0, '-': 0, '--': 0, '-=': 0, '/': 0, '//': 0, '/=': 0, ':': 2, ';': 0, '<': 2, '<<': 5, '<<=': 0, '<=': 0, '=': 2, '==': 0, '>': 0, '>=': 0, '>>': 4, '>>=': 0, '?': 2, '[': 0, ']': 0, '^': 0, '^=': 0, '|': 0, '|=': 0, '||': 0, '~': 0}
